## Equal-Weight S&P 500 Index Fund

The purpose of this program Python script is that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [5]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [6]:
from secrets import IEX_CLOUD_API_TOKEN

https://iexcloud.io/docs/api/ (iex documentation link)
search for sandbox in iex docs and 
copy url (https://sandbox.iexapis.com) to use instead of base url.

We need the following information from the API:
     1)Market capitalization for each stock
     2)Price of each stock
If you use the "quote" endpoint, it will give you both. Search "quote and copy endpoint "/stock/{symbol}/quote/{field} " under HTTP request to later modify. 

The API URL was changed to an f-string to change our base API url "...{symbol}...."

I added an appendage to the end of this API URL (?token={IEX_CLOUD_API_TOKEN}') that passes in our API token to authenticate us through the iex cloud API to, in other words, say that i have permission to use this API endpoint.

"/stable" is used to make sure your application never breaks. also "/latest" for latest leading edge tech. However, not useful for this. 

Now we have a iex cloud endpoint that we can ping using the request library.


In [7]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
## to test. should be equal to True. 
#data = requests.get(api_url)
#print(data.status_code == 200)
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'QSG BTR NDK()GO/EESAALLLACNM EAST',
 'calculationPrice': 'close',
 'open': 134.67,
 'openTime': 1691071606581,
 'openSource': 'oiaifclf',
 'close': 132.66,
 'closeTime': 1620158995495,
 'closeSource': 'fcoafili',
 'high': 131.57,
 'highTime': 1661784176231,
 'highSource': 'e5 ieacp1ddyutee lr nim',
 'low': 133.7,
 'lowTime': 1628300151382,
 'lowSource': 'pitr 1m leu edenydiea5c',
 'latestPrice': 132.84,
 'latestSource': 'Close',
 'latestTime': 'February 19, 2021',
 'latestUpdate': 1626970955679,
 'latestVolume': 91250163,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 130.57,
 'delayedPriceTime': 1654990938582,
 'oddLotDelayedPrice': 131.947,
 'oddLotDelayedPriceTime': 1693566075304,
 'extendedPrice': 134.6,
 'extendedChange': -0.16,
 'extendedChangePercent': -0.00117,
 'extendedPriceTime': 1613975291431,
 'previousClose': 131.31,
 'previousVolume': 98107011,
 'change': 

Note: in JSON file, if you're not sure what a variable is, copy paste and search in iex docs. There are descriptions.

In [8]:
data['latestPrice']
data['marketCap']

2235530405922

In [9]:
# check
#price1 = data['latestPrice']
#price2 = data['marketCap']
#print(price1)
#devide by a trillion
#print(price2/1000000000000)


Add our stocks to our Pandas datafrman

In [10]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe1 = pd.DataFrame([[0,0,0,0]],columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


Appending data points to pandas dataframe.
Creating a pandas series that lists all of the entries in the dataframe. A pandas series is a one dimensional dataframe. Unlike a padas dataframe, which is 2 dimensional. Whithin a pandas dataframe each row and columns in a datframe is a pandas series. In order to append data to a pandas dataframe, that data needs to be a pandas series. 

"index = my_columns" tells the append method wich columns to add that data to. Otherwise will have double the columns with NaN values in your dataframe. Argument needs to be within padas series and not within dataframe.

In [11]:
# Must add "ignore_index = True" when appending data to a pandas dataframe.
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,132.84,2235530405922,N/A


Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.

We need to loop through every ticker in our stocks varibale and execute and API call for that stock and store the results in our pandas dataframe.

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
    #the above method works but take a while to execute. 
    #We will overwrite the final_dataframe with a faster but more comlpicated method.    

KeyboardInterrupt: 

In [37]:
  #print(stocks)

Note: In regards to the above code, "for symbol in stocks['Ticker'][:5]:" requests the first five stocks. Also in general "final_dataframe.append(..."  won't modify the original dataframe unless you tell it to. Therefore, we write " final_dataframe = final_dataframe.append(..."  

Batch API calls are one of the easiest ways to improve the performance of your code.
This is because HTTP requests are typically one of the slowest components of a script.
Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.
IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

What we need to do is split our list into lists of length 100. Google "how to split lists into sub lists in python" most codes will work just fine.  

In [13]:
# Function from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

code:"symbol_groups = list(chunks(stocks['Ticker'], 100))"
Thge reason why we have to pass "ticker" is bc it is a pandas dataframe and ticker is the header of the only column. 
Note: if we just did "stocks" without ticker, it would give us the dataframe. Using header "ticker" , ie. stocks['Ticker'], gives us a series that we can pass into the chuncks function.

The for loop that loops through every pandas series within that list; executes a batch API call and then for every stock in that list appends that information from that stock to our final dataframe. This code prints te list of lists within our symbol group:
for i in range(0, len(symbol_groups)):
     print(symbol_strings[i])

We need to transform all of the stocks that are in each of those lists into a string and that string and that string will be passed into the url of the http request that we are executing.


Note: can go to iex docs for  batch requests. URL code from iex:  
".../stock/market/batch?symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5 "
Note: some parameters not applicable.also after"?" every symbol thereafter needs to be "&".
final code for f string: https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
reminder: "...symbol_strings[:1]:" makes it loop over first instance or batch (100).     



In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = [] # a list of strings where each string is a comma seperated string of all the stocks in this object.
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
#Note: final_dataframe is being overwitten 
final_dataframe = pd.DataFrame(columns = my_columns)

#looping through symbol_strings object and use that string to execute our batch API request.  
for symbol_string in symbol_strings:
#     print(symbol_strings)
# passing in the base url of the API endpoint and then pass in all of the strings for each the symbol strings 
#  into the ednpoint to get data from a 100 stocks back.
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #parsing data for every stock that is in this symbol string.
    for symbol in symbol_string.split(','):
        #print(symbol)
        #parsing batch API call for data for stock in order and append in final_dataframe.
        # as well as metric for the stock. "....symbol,........"
        #N/A last columns will be added later
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,129.15,39473291446,N/A
1,AAL,19.30,12356021776,N/A
2,AAP,173.67,11454886546,N/A
3,AAPL,135.02,2286264253817,N/A
4,ABBV,105.91,186379433880,N/A
...,...,...,...,...
500,YUM,106.41,31995654987,N/A
501,ZBH,162.02,32563271362,N/A
502,ZBRA,496.70,26669773352,N/A
503,ZION,54.20,8611804904,N/A


final_dataframe

In [44]:
#check above "data" renamed data1. 200 means it's working properly for our purposes.  
#data1 = requests.get(batch_api_call_url)
#print(data1.status_code)

Calculating the Number of Shares to Buy
Based on portfolio size it will tell you the number of shares to buy.

In [15]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


Equal weight is distributed to the the purchase of stocks based on value. Portfolio value will be distributed accross all stocks. 

The "final_dataframe.loc[i, 'Number Of Shares to Buy']..." is an easy way to access the last row that was N/A in our dataframe. "i" is the row"Number of share to buy is the column".  

In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,129.15,39473291446,153
1,AAL,19.30,12356021776,1026
2,AAP,173.67,11454886546,114
3,AAPL,135.02,2286264253817,146
4,ABBV,105.91,186379433880,186
...,...,...,...,...
500,YUM,106.41,31995654987,186
501,ZBH,162.02,32563271362,122
502,ZBRA,496.70,26669773352,39
503,ZION,54.20,8611804904,365


In [17]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


Applying the Formats to the Columns of Our .xlsx File
We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself".
Let's simplify this by putting it in 2 loops:

In [21]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [22]:
writer.save()